In [1]:
import torch
print(torch.__version__)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

1.1.0


In [2]:
batch_size = 128
learning_rate = 0.001
num_epochs = 20

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = F.dropout(out, 0.2, self.training)
        out = self.fc2(out)
        out = F.relu(out)
        out = F.dropout(out, 0.2, self.training)
        out = self.fc3(out)
        return out

model = NeuralNet().to(device)

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/20], Step [100/469], Loss: 0.4074
Epoch [1/20], Step [200/469], Loss: 0.1880
Epoch [1/20], Step [300/469], Loss: 0.1706
Epoch [1/20], Step [400/469], Loss: 0.1473
Epoch [2/20], Step [100/469], Loss: 0.0756
Epoch [2/20], Step [200/469], Loss: 0.1606
Epoch [2/20], Step [300/469], Loss: 0.0896
Epoch [2/20], Step [400/469], Loss: 0.0632
Epoch [3/20], Step [100/469], Loss: 0.0487
Epoch [3/20], Step [200/469], Loss: 0.0508
Epoch [3/20], Step [300/469], Loss: 0.0589
Epoch [3/20], Step [400/469], Loss: 0.1442
Epoch [4/20], Step [100/469], Loss: 0.0201
Epoch [4/20], Step [200/469], Loss: 0.0786
Epoch [4/20], Step [300/469], Loss: 0.0524
Epoch [4/20], Step [400/469], Loss: 0.0132
Epoch [5/20], Step [100/469], Loss: 0.0267
Epoch [5/20], Step [200/469], Loss: 0.0363
Epoch [5/20], Step [300/469], Loss: 0.0651
Epoch [5/20], Step [400/469], Loss: 0.0277
Epoch [6/20], Step [100/469], Loss: 0.0380
Epoch [6/20], Step [200/469], Loss: 0.0352
Epoch [6/20], Step [300/469], Loss: 0.0733
Epoch [6/20

In [7]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.34 %
